In [50]:
from langchain.prompts import PromptTemplate

hallucination_grader_prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary score 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)


In [52]:
from langchain_groq import ChatGroq
from langchain_core.pydantic_v1 import BaseModel, Field

grader_llm_name = "llama3-70b-8192"


class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )



grader_llm = ChatGroq(
    model=grader_llm_name,
    temperature=0,
)

hallucination_grader = hallucination_grader_prompt | grader_llm.with_structured_output(GradeHallucinations)

In [54]:
documents = """the INS OCT -NS7300 -01A, GNSS Antenna, and associated cable harness. This manual provides 
instructions for the scheduled maintenance manual for OCT -NS7300 -01A and  GNSS Antenna as 
installed in an armoured vehicle. The installation of the INS system and associated accessories is 
performed in accordance with their respective Technical Solution Manual.  
Section 2  Maintenance Intervals  
The equipment will be maintained at the following Maintenance echelons:  
● O Level Maintenance (Organization Maintenance) - by the Armoured Corps technicians.  
● I Level Maintenance (Intermediate Maintenance) - by the Armoured Corps technicians  
● D Level Maintenance (Depot Maintenance) - by the Aeron technicians  
 
The maintenance activities to be performed in O - and I -Level, consist of both periodic/preventive 
activities, as well as corrective activities.  
Maintenance tasks, principles, methods and aids are described in the following sub -paragraphs  
O Level Maintenance (Organization Maintenance) ● Base Functional Test such as, current consumption, System Health Status  
● Initiate the Raw Data to check the data readiness of the OCT -2 NS7300D 01A unit.  
● Review the satellite constellation status in GPS View  
● Technician’s Review of the actual software and firmware versions.  
 
I-Level Test Equipment (ILTE)  
The ILTE is used in maintenance facilities, enables the Armoured Corps maintenance units to 
perform maintenance activities on vehicles. During the ILTE is connected (over Moxa Uport 
1150 & RS422).  
● Initiate a review of the position data of the MEMS INS unit.  
● Check the Power supply or check the voltage level of the power source.  
● Conduct a visual check of harness bonding and cable cover/conductive tape used for 
electric connector bonding.  
● Replace any damaged or torn straps.  
● Check Corrosion on the INS unit, GNSS antenna connector, or cable harness winding. RESTRICTED  
 
7 Chapter 3  Chapter III - Fault Diagnosis  
Section 1  Troubleshooting  
This section provides information to assist in troubleshooting if a problem occurs after 
completing the installation or maintenance. The following table will be used on the I -Level 
maintenance intervals.  When troubleshooting the MEMS INS system, refer to the drawings and 
User Manual that are retained in the permanent records.  
 
Symptom  Possible Cause  Corrective Action  
The Inertial status not 
showing in the software 
utility  The INS unit is not 
powered on.  Check the power supply switch of the 
INS unit. Ensure the power supply 12 
pin connector. Increase the input 
power supply to 28V .  
The GNSS does not 
compute a position  Not receiving adequate 
GNSS signals.  Check the GNSS antenna 
connections, and make sure the 
GNSS antenna is clear of 
obstructions (hangars, buildings, 
trees, etc.) RESTRICTED  
 
iii LIST OF ASSOCIATED HANDBOOKS  
Technical Literature for MEMS INS SYSTEM  
(Main Equipment/ System) comprises of the followings:  
USER HANDBOOK AND INSTRUCTIONS FOR INSTALLATION INCLUDING 
CHECKLIST  
TECHNICAL MANUAL  
PART I TECHNICAL INFORMATION  
V olume 1 Technical Description  
V olume 2 Drawings  
PART II MAINTENANCE  
PART III OVERHAUL AND RECONDITIONING INSTRUCTIONS  
PART IV MANUFACTURER'S PARTS LIST  
V olume 1 Parts List  
V olume 2 Illustrations"""


generation = """The maintenance procedures for an INS (Inertial Navigation System) include O-Level Maintenance, I-Level Maintenance, and D-Level Maintenance. O-Level Maintenance is performed by Armoured Corps technicians, I-Level Maintenance is performed by Armoured Corps technicians, and D-Level Maintenance is performed by Aeron technicians."""


score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )

print(score)

score='yes'
